In [3]:
import pandas as pb
data = pb.read_excel('updFAKSexp.xlsx')
print(1)
#data.to_csv('updFAKSexpCSV.csv')
data_np = data.to_numpy()
print(data.shape)

1
(129730, 487)


In [6]:
from sklearn.model_selection import train_test_split
import pandas as pb

input_tokens = [(input_text) for input_text in data.iloc[:, 1:].iterrows()]
print(len(input_tokens))
output_tokens = [(output_text) for output_text in data.iloc[:, 0]]

par_tokens = [(output_tokens[i], input_tokens[i])for i in range(len(input_tokens))]

train_texts, val_and_test_texts = train_test_split(par_tokens, test_size=0.2, random_state=42) #måste läsa lite om random_state
val_texts, test_texts = train_test_split(val_and_test_texts, test_size=0.5, random_state=42)

output_tokens_from_par_train = [item[0] for item in train_texts]
output_tokens_from_par_val = [item[0] for item in val_texts]
output_tokens_from_par_test = [item[0] for item in test_texts]
input_tokens_from_par_train = [item[1:] for item in train_texts]
input_tokens_from_par_val = [item[1:] for item in val_texts]
input_tokens_from_par_test = [item[1:] for item in test_texts]

PatientJournalText_train_small = [str(item) for item in output_tokens_from_par_train[:80]]
PatientJournalText_val_small = [str(item) for item in output_tokens_from_par_val[:10]]
PatientJournalText_test_small = [str(item) for item in output_tokens_from_par_test[:10]]
InputData_train_small = [str(item) for item in input_tokens_from_par_train[:80]]
InputData_val_small = [str(item) for item in input_tokens_from_par_val[:10]]
InputData_test_small = [str(item) for item in input_tokens_from_par_test[:10]]

129730


In [7]:
import torch
from transformers import PreTrainedTokenizerFast, pipeline, AutoTokenizer, BioGptTokenizer, BioGptForCausalLM
import os

from accelerate import Accelerator

accelerator = Accelerator(cpu=True)
# Set environment variables
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

class MyDataset(torch.utils.data.Dataset):
    def __init__(self, PatientJournalText_small, InputData_small, tokenizer):
        self.PatientJournalText_small = PatientJournalText_small
        self.InputData_small = InputData_small
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.InputData_small)

    def __getitem__(self, idx):        
        PatientJournalText_small = self.PatientJournalText_small[idx]
        InputData_small = self.InputData_small[idx]

        # Encode input sequence and labels using tokenizer
        inputs = self.tokenizer.encode_plus(
            InputData_small,
            return_tensors='pt',
            padding='max_length',
            truncation=True,
            max_length=512,  # Set this according to your maximum sequence length
        )

        labels = self.tokenizer.encode_plus(
            PatientJournalText_small,
            return_tensors='pt',
            padding='max_length',
            truncation=True,
            max_length=512,  # Set this according to your maximum sequence length
        )

        return {
            "input_ids": inputs.input_ids.flatten(),
            "labels": labels.input_ids.flatten()
        }

# Assuming you have already initialized the tokenizer somewhere
#device = "cuda:0" if torch.cuda.is_available() else "cpu"
model_name = "microsoft/biogpt"
tokenizer = BioGptTokenizer.from_pretrained(model_name)
model = BioGptForCausalLM.from_pretrained(model_name)
#model.to(device)

AllData_train_small = MyDataset(PatientJournalText_train_small, InputData_train_small, tokenizer)
AllData_val_small = MyDataset(PatientJournalText_val_small, InputData_val_small, tokenizer)
AllData_test_small = MyDataset(PatientJournalText_test_small, InputData_test_small, tokenizer)

In [23]:
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForCausalLM
import numpy as np
from accelerate import Accelerator
import evaluate
from accelerate import Accelerator

#accelerator = Accelerator(cpu=True)



training_args = TrainingArguments(
    output_dir="test_trainer", 
    evaluation_strategy="epoch",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    logging_steps=1000,
    save_steps=2000,
    logging_dir="./logs")
    #metric = evaluate.load("accuracy")

'''def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)'''

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset= AllData_train_small,
    eval_dataset= PatientJournalText_val_small,
    #compute_metrics= compute_metrics,
)

trainer.train()


  0%|          | 0/20 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB. GPU 0 has a total capacity of 5.00 GiB of which 0 bytes is free. Of the allocated memory 1.45 GiB is allocated by PyTorch, and 86.21 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [24]:
import torch
torch.cuda.empty_cache()
torch.cuda.memory_allocated()

1551676928